# Write Experiments

This file is used to generate JSON files for running experients

JSON files will be written for main.py to read arguments from, rather than going from the command-line

In [1]:
import pickle
import pandas as pd
from datetime import datetime
import os

## Experiment 1: Number of Objects, dist vs goal

Basic toy example, each 

In [33]:
num_objs = [7]
rew_structures = ['dist', 'goal']


for rew_struct in rew_structures:
    for num_obj in num_objs:
        exp_name = f'nav_{num_obj}_obj_{rew_struct}_recurrent'
        config = {
            'wandb_project_name': 'nav1',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 250000,
            'env_name': 'NavEnv-v0',
            'recurrent_policy': True,
            'algo': 'ppo',
            'num_mini_batch': 16,
            'num_processes': 16,
            'env_kwargs': {'num_objects': num_obj, 'rew_structure': rew_struct},
            'track': True,
            'capture_video': 1
        }

        pickle.dump(config, open('experiment_configs/' + exp_name, 'wb'))

In [8]:
num_objs = [0, 1, 2, 3, 4]
rew_structures = ['dist', 'goal']


for rew_struct in rew_structures:
    for num_obj in num_objs:
        exp_name = f'nav_{num_obj}_obj_{rew_struct}'
        config = {
            'wandb_project_name': 'nav1',
            'exp_name': exp_name,
            'save_name': exp_name,
            'num_env_steps': 250000,
            'env_name': 'NavEnv-v0',
            'algo': 'ppo',
            'num_mini_batch': 16,
            'num_processes': 16,
            'env_kwargs': {'num_objects': num_obj, 'rew_structure': rew_struct},
            'track': True,
            'capture_video': 1
        }

        pickle.dump(config, open('experiment_configs/' + exp_name, 'wb'))

In [4]:
exp_name = 'nav_0_obj_goal'
config = {
    'wandb_project_name': 'nav1',
    'exp_name': exp_name,
    'save_name': exp_name,
    'num_env_steps': 250000,
    'env_name': 'NavEnv-v0',
    'recurrent_policy': True,
    'algo': 'ppo',
    'num_mini_batch': 4,
    'num_processes': 4,
    'env_kwargs': {'num_objects': 0},
    'track': True,
    'capture_video': True
}

pickle.dump(config, open('experiment_configs/' + exp_name, 'wb'))

In [18]:
pickle.load(open('experiment_configs/nav_0_obj', 'rb'))

{'wandb_project_name': 'nav1',
 'exp_name': 'nav_0_obj',
 'save_name': 'nav_0_obj',
 'num_env_steps': 10000,
 'env_name': 'NavEnv-v0',
 'recurrent_policy': True,
 'algo': 'ppo',
 'num_mini_batch': 4,
 'num_processes': 4,
 'env_kwargs': {'num_objects': 0},
 'track': True,
 'capture_video': True}

In [9]:
files = os.listdir('experiment_configs/')

In [10]:
for file in files:
    if file not in ['.ipynb_checkpoints', 'archive']:
        print(file)

hello_world


In [16]:
pickle.load(open('experiment_log', 'rb'))

,file,begin,end,exp_name,save_name,num_env_steps,env_name,recurrent_policy,algo,num_mini_batch,num_processes,success,env_kwargs,wandb_project_name
0,hello_world,2021-10-25 18:23:03.197613,NaN,hello_world,hello_world,10000,CartPole-v0,True,ppo,4,4,NaN,NaN,NaN
1,hello_world,2021-10-25 18:28:38.870516,2021-10-25 18:43:41.004250,hello_world,hello_world,10000,CartPole-v0,True,ppo,4,4,NaN,NaN,NaN
2,hello_world,2021-10-25 18:58:10.351818,2021-10-25 18:59:07.591750,hello_world,hello_world,10000,CartPole-v0,True,ppo,4,4,True,NaN,NaN
3,nav_0_obj,2021-10-27 13:05:19.228026,NaN,nav_0_obj,nav_0_obj,10000,NavEnv-v0,True,ppo,4,4,NaN,{'num_objects': 0},nav1


## Scheduler functions

These functions are found in scheduler.py (testing them here), used to run and track the experiments in experiment_configs

In [3]:
from scheduler import *
import torch

In [3]:
js = pickle.load(open('exp_log.json', 'rb'))

In [6]:
df = pd.read_json(js)

In [7]:
df.to_pickle('experiment_log')

In [5]:
run_experiment('morris_epstruct_1_recurrent_sep_false', cont=True)

In [4]:
convert_config_to_command('morris_epstruct_1_recurrent_sep_false', cont=True)

'python main.py --wandb-project-name nav2 --exp-name morris_epstruct_1_recurrent_sep_false --save-name morris_epstruct_1_recurrent_sep_false --num-env-steps 500000 --env-name MorrisEnv-v0 --recurrent-policy --algo ppo --num-mini-batch 16 --num-processes 16 --env-kwargs ep_struct=1 character_sep=False --track --capture-video 1 --cont --config-file-name morris_epstruct_1_recurrent_sep_false '

In [6]:
actor_critic, obs_rms = torch.load('trained_models/ppo/morris_epstruct_1_recurrent_sep_false.pt')

In [9]:
int(obs_rms.count)

192096

In [10]:
from a2c_ppo_acktr import algo, utils


In [15]:
algo.PPO(actor_critic, 1.2, 100, 16, 0.1, 0.1, lr=0.005, eps=0.01, )

In [17]:
actor_critic.recurrent_hidden_state_size

64

In [3]:
run_experiment('hello_world')

# Morris Environment Experiments

In [18]:
ep_structs = [1, 2]


for ep_struct in ep_structs:
    exp_name = f'morris_epstruct_{ep_struct}_recurrent_sep_false'
    config = {
        'wandb_project_name': 'nav2',
        'exp_name': exp_name,
        'save_name': exp_name,
        'num_env_steps': 500000,
        'env_name': 'MorrisEnv-v0',
        'recurrent_policy': True,
        'algo': 'ppo',
        'num_mini_batch': 2,
        'num_processes': 2,
        'env_kwargs': {'ep_struct': ep_struct, 'character_sep': False},
        'track': True,
        'capture_video': 1
    }

    pickle.dump(config, open('experiment_configs/' + exp_name, 'wb'))